### 请仔细阅读DCGAN相关材料并补充完整下面的代码。在需要补充的部分已经标注#TODO并附上相应的内容提示。

In [2]:
# 导入必要的库
import torch  # PyTorch 深度学习框架
import torch.nn as nn  # 神经网络相关模块
import numpy as np  # 数值计算库
from torch.utils.data import DataLoader  # 处理数据加载
import torchvision
from torchvision import datasets, transforms  # 处理图像数据集和数据变换
from torchvision.utils import save_image  # 保存生成的图像
import os  # 处理文件和目录操作
from torch.utils.tensorboard import SummaryWriter  # TensorBoard

#### 根据文档和提示，参考GAN的实现代码，补充完整DCGAN的生成器Generator和判别器Discriminator代码：

In [3]:
# =============================== 生成器（Generator） ===============================
class Generator(nn.Module):
    def __init__(self, input_dim):
        super(Generator, self).__init__()

        # 1. 输入层：将 100 维随机噪声投影到 32x32（1024 维）
        #TODO   # 线性变换fc1，将输入噪声扩展到 1024 维
        self.fc1 = nn.Linear(input_dim, 32*32)
        self.br1 = nn.Sequential(
            #TODO   # 批归一化，加速训练并稳定收敛
            nn.BatchNorm1d(32*32),
            #TODO   # ReLU 激活函数，引入非线性
            nn.ReLU()
        )

        # 2. 第二层：将 1024 维数据映射到 128 * 7 * 7 维特征
        #TODO   # 线性变换fc2，将数据变换为适合卷积层的维数大小
        self.fc2 = nn.Linear(32*32, 128*7*7)
        self.br2 = nn.Sequential(
            #TODO   # 批归一化
            nn.BatchNorm1d(128*7*7),
            #TODO   # ReLU 激活函数
            nn.ReLU()
        )

        # 3. 反卷积层 1：上采样，输出 64 通道的 14×14 特征图
        self.conv1 = nn.Sequential(
            #TODO   # 反卷积：将 7x7 放大到 14x14，kernel size设置为4，stride设置为2，padding设置为1
            nn.ConvTranspose2d(in_channels=128 ,out_channels=64,kernel_size=4, stride=2, padding=1),
            #TODO   # 归一化，稳定训练
            nn.BatchNorm2d(64),
            #TODO   # ReLU 激活函数
            nn.ReLU()
        )

        # 4. 反卷积层 2：输出 1 通道的 28×28 图像
        self.conv2 = nn.Sequential(
            #TODO   # 反卷积：将 14x14 放大到 28x28，将 7x7 放大到 14x14，kernel size设置为4，stride设置为2，padding设置为1
            nn.ConvTranspose2d(in_channels=64 ,out_channels=1 ,kernel_size=4, stride=2, padding=1),
            #TODO   # Sigmoid 激活函数，将输出归一化到 [0,1]
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.br1(self.fc1(x))  # 通过全连接层，进行 BatchNorm 和 ReLU 激活
        x = self.br2(self.fc2(x))  # 继续通过全连接层，进行 BatchNorm 和 ReLU 激活
        x = x.reshape(-1, 128, 7, 7)  # 变形为适合卷积输入的形状 (batch, 128, 7, 7)
        x = self.conv1(x)  # 反卷积：上采样到 14x14
        output = self.conv2(x)  # 反卷积：上采样到 28x28
        return output  # 返回生成的图像

# =============================== 判别器（Discriminator） ===============================
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        # 1. 第一层：输入 1 通道的 28x28 图像，输出 32 通道的特征图，然后通过MaxPool2d降采样
        self.conv1 = nn.Sequential(
            #TODO  # 5x5 卷积核，步长为1
            nn.Conv2d(in_channels=1, out_channels=32 ,kernel_size=5, stride=1),
            #TODO   # LeakyReLU，negative_slope参数设置为0.1
            nn.LeakyReLU(negative_slope=0.1)
        )
        self.pl1 = nn.MaxPool2d(2, stride=2)

        # 2. 第二层：输入 32 通道，输出 64 通道特征, 然后通过MaxPool2d降采样
        self.conv2 = nn.Sequential(
            #TODO   # 5x5 卷积核，步长为1
            nn.Conv2d(in_channels=32, out_channels=64 ,kernel_size=5, stride=1),
            #TODO  # LeakyReLU 激活函数，negative_slope参数设置为0.1
            nn.LeakyReLU(negative_slope=0.1)
        )
        self.pl2 = nn.MaxPool2d(2, stride=2)

        # 3. 全连接层 1：将 64x4x4 维特征图转换成 1024 维向量
        self.fc1 = nn.Sequential(
            #TODO   # 线性变换，将 64x4x4 映射到 1024 维
            nn.Linear(64*4*4, 1024),
            #TODO   # LeakyReLU 激活函数，negative_slope参数设置为0.1
            nn.LeakyReLU(negative_slope=0.1)
        )

        # 4. 全连接层 2：最终输出真假概率
        self.fc2 = nn.Sequential(
            #TODO   # 线性变换，将 1024 维数据映射到 1 维
            nn.Linear(1024,1),
            #TODO   # Sigmoid 归一化到 [0,1] 作为概率输出
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.pl1(self.conv1(x))  # 第一层卷积，降维
        x = self.pl2(self.conv2(x))  # 第二层卷积，降维
        x = x.view(x.shape[0], -1)  # 展平成向量
        x = self.fc1(x)  # 通过全连接层
        output = self.fc2(x)  # 通过最后一层全连接层，输出真假概率
        return output  # 返回判别结果

#### 补充完整主函数，在主函数中完成以下过程：
1. 数据加载：
加载并预处理数据集。对于DCGAN的训练，通常需要将数据集转换为张量格式，并进行适当的归一化。
2. 模型初始化：
创建生成器和判别器模型实例，并将它们移动到合适的设备（如GPU）上。
3. 优化器和损失函数定义：
为生成器和判别器分别定义优化器（如Adam），并设置适当的学习率和其他超参数。
定义损失函数（如二元交叉熵损失）用于评估模型性能。
4. 训练循环：
迭代多个epoch进行训练。在每个epoch中，遍历数据集并进行以下操作：
   * 训练判别器：使用真实数据和生成的假数据更新判别器的参数。
   * 训练生成器：通过生成假数据并试图欺骗判别器来更新生成器的参数。
   * 记录损失值到TensorBoard，以监控训练过程。
5. 结果保存：
在每个epoch结束时，生成一些示例图像并保存到TensorBoard，以便观察生成器的进展。

In [4]:
# =============================== 主函数 ===============================
def main():
    # 设备配置：使用 GPU（如果可用），否则使用 CPU
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # 设定超参数
    input_dim = 100  # 生成器输入的随机噪声向量维度
    batch_size = 128  # 训练时的批量大小
    num_epoch = 10  # 训练的总轮数

    # 加载 MNIST 数据集
    train_dataset = datasets.MNIST(root="./data/", train=True, transform=transforms.ToTensor(), download=True)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

    # 创建生成器和判别器，并移动到 GPU（如果可用）
    # TODO
    G = Generator(input_dim).to(device)
    # TODO
    D = Discriminator().to(device)

    # 定义优化器，优化器要求同任务一
    # TODO
    optim_G = torch.optim.Adam(G.parameters(), lr=0.0002)
    # TODO
    optim_D = torch.optim.Adam(D.parameters(), lr=0.0002)
    loss_func = nn.BCELoss()

    # 初始化 TensorBoard
    writer = SummaryWriter(log_dir='./logs/experiment_dcgan')

    # 开始训练
    for epoch in range(num_epoch):
        total_loss_D, total_loss_G = 0, 0
        for i, (real_images, _) in enumerate(train_loader):
            loss_D = train_discriminator(real_images, D, G, loss_func, optim_D, batch_size, input_dim, device)
            loss_G = train_generator(D, G, loss_func, optim_G, batch_size, input_dim, device)

            total_loss_D += loss_D
            total_loss_G += loss_G

            # 每 100 步打印一次损失
            if (i + 1) % 100 == 0 or (i + 1) == len(train_loader):
                print(f'Epoch {epoch:02d} | Step {i + 1:04d} / {len(train_loader)} | Loss_D {total_loss_D / (i + 1):.4f} | Loss_G {total_loss_G / (i + 1):.4f}')

        # 记录损失到 TensorBoard
        writer.add_scalar('DCGAN/Loss/Discriminator', total_loss_D / len(train_loader), epoch)
        writer.add_scalar('DCGAN/Loss/Generator', total_loss_G / len(train_loader), epoch)

        # 生成并保存示例图像
        with torch.no_grad():
            noise = torch.randn(64, input_dim, device=device)
            fake_images = G(noise)

            # 记录生成的图像到 TensorBoard
            img_grid = torchvision.utils.make_grid(fake_images, normalize=True)
            writer.add_image('Generated Images', img_grid, epoch)

    writer.close()

#### 根据文档中描述的GAN的损失函数和二元交叉熵损失相关内容，补充完善Discriminator和Generator的训练过程：

In [5]:
# =============================== 训练判别器 ===============================
def train_discriminator(real_images, D, G, loss_func, optim_D, batch_size, input_dim, device):
    '''训练判别器'''
    real_images = real_images.to(device)
    real_output = D(real_images) # 判别器预测真实图像
    real_target = real_images.mean(dim = [2,3])
    #TODO   # 计算真实样本的损失real_loss
    real_loss = loss_func(real_output, real_target)
    noise = torch.randn(real_images.shape[0], input_dim, device=device)  # 生成随机噪声
    fake_images = G(noise).detach()  # 生成假图像（detach 避免梯度传递给 G）
    fake_output = D(fake_images)  # 判别器预测假图像
    
    #TODO   # 计算假样本的损失fake_loss
    fake_loss = loss_func(fake_output, real_target)

    loss_D = real_loss + fake_loss  # 判别器总损失
    optim_D.zero_grad()  # 清空梯度
    loss_D.backward()  # 反向传播
    optim_D.step()  # 更新判别器参数

    return loss_D.item()  # 返回标量损失


# =============================== 训练生成器 ===============================
def train_generator(D, G, loss_func, optim_G, batch_size, input_dim, device):
    '''训练生成器'''
    noise = torch.randn(batch_size, input_dim, device=device).to(device)  # 生成随机噪声
    fake_images = G(noise)  # 生成假图像
    fake_output = D(fake_images)  # 判别器对假图像的判断
    #TODO # 计算生成器损失（希望生成的图像判别为真）
    loss_G = loss_func(fake_output, torch.ones(batch_size,1).to(device))
    optim_G.zero_grad()  # 清空梯度
    loss_G.backward()  # 反向传播
    optim_G.step()  # 更新生成器参数

    return loss_G.item()  # 返回标量损失

#### 主函数执行入口

In [6]:
if __name__ == '__main__':
    main()

Epoch 00 | Step 0100 / 469 | Loss_D 0.8020 | Loss_G 1.9363
Epoch 00 | Step 0200 / 469 | Loss_D 0.7838 | Loss_G 1.9886
Epoch 00 | Step 0300 / 469 | Loss_D 0.7784 | Loss_G 2.0043
Epoch 00 | Step 0400 / 469 | Loss_D 0.7757 | Loss_G 2.0119
Epoch 00 | Step 0469 / 469 | Loss_D 0.7748 | Loss_G 2.0151
Epoch 01 | Step 0100 / 469 | Loss_D 0.7724 | Loss_G 2.0677
Epoch 01 | Step 0200 / 469 | Loss_D 0.7706 | Loss_G 2.0531
Epoch 01 | Step 0300 / 469 | Loss_D 0.7696 | Loss_G 2.0483
Epoch 01 | Step 0400 / 469 | Loss_D 0.7695 | Loss_G 2.0442
Epoch 01 | Step 0469 / 469 | Loss_D 0.7688 | Loss_G 2.0437
Epoch 02 | Step 0100 / 469 | Loss_D 0.7671 | Loss_G 2.0308
Epoch 02 | Step 0200 / 469 | Loss_D 0.7688 | Loss_G 2.0167
Epoch 02 | Step 0300 / 469 | Loss_D 0.7690 | Loss_G 2.0079
Epoch 02 | Step 0400 / 469 | Loss_D 0.7695 | Loss_G 1.9959
Epoch 02 | Step 0469 / 469 | Loss_D 0.7697 | Loss_G 1.9905
Epoch 03 | Step 0100 / 469 | Loss_D 0.7711 | Loss_G 1.9588
Epoch 03 | Step 0200 / 469 | Loss_D 0.7707 | Loss_G 1.95